In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error
import os
import seaborn as sns
from sklearn import metrics, tree, ensemble
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
import statsmodels.formula.api as smf
import statsmodels.api as sm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/data_final5.csv"

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
df['building_type'] = df['building_type'].astype('category')

In [ ]:
datatrain = df.iloc[:393588]
datatest = df.iloc[393588:491986]

In [ ]:
# Разделение данных на тренировочный и тестовый наборы
X_train = datatrain[['geo_lon', 'geo_lat', 'building_type', 'level', 'levels', 'rooms',
        'area', 'kitchen_area', 'object_type', 'distance_to_cemetery',
        'distance_to_attractions', 'distance_to_green_zone',
        'distance_to_metro_station', 'distance_to_polygon', 'KeyRate']]
y_train = datatrain['priceL']

X_test = datatest[['geo_lon', 'geo_lat', 'building_type', 'level', 'levels', 'rooms',
        'area', 'kitchen_area', 'object_type', 'distance_to_cemetery',
        'distance_to_attractions', 'distance_to_green_zone',
        'distance_to_metro_station', 'distance_to_polygon', 'KeyRate']]
y_test = datatest['price']

a = 98398

#train (new)
X_trainn = X_train[:-a]
y_trainn = y_train[:-a]
#valid
X_trainV = X_train[-a:]
y_trainV = y_train[-a:]

In [ ]:
num_features = list(X_train.columns)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features)
    ])

X_trainN_norm = preprocessor.fit_transform(X_trainn)
X_test_norm = preprocessor.transform(X_test)
X_trainV_norm = preprocessor.transform(X_trainV)
X_train.shape, X_test.shape

((393588, 15), (98398, 15))

In [ ]:

# Создание модели
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(1, activation='relu'))

# Компиляция модели
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mape'])


os.makedirs("models", exist_ok=True)

# создаем чекпойнты для сохранения модели и редьюс он плато схедуллер
checkpoint = keras.callbacks.ModelCheckpoint("models/model.keras", verbose=0,
                                             save_best_only=True, monitor='val_mape',
                                             save_weights_only=False, mode='auto')


reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                              patience=5, min_lr=0.01/100)


In [ ]:
# тренируем модель
history = model.fit(
    X_trainN_norm, y_trainn,
    batch_size=256,
    epochs=100,
    verbose=1,
    validation_data=(X_trainV_norm, y_trainV), callbacks=[checkpoint, reduce_lr])

Epoch 1/100
1154/1154 [==============================] - 13s 8ms/step - loss: 16.2182 - mape: 18.8225 - val_loss: 0.5542 - val_mape: 3.9997 - lr: 0.0010
Epoch 2/100
1154/1154 [==============================] - 7s 6ms/step - loss: 6.8791 - mape: 13.1048 - val_loss: 0.5834 - val_mape: 4.2494 - lr: 0.0010
Epoch 3/100
1154/1154 [==============================] - 5s 5ms/step - loss: 5.3481 - mape: 11.5386 - val_loss: 0.1810 - val_mape: 2.1555 - lr: 0.0010
Epoch 4/100
1154/1154 [==============================] - 6s 5ms/step - loss: 4.2745 - mape: 10.3891 - val_loss: 0.0867 - val_mape: 1.4295 - lr: 0.0010
Epoch 5/100
1154/1154 [==============================] - 7s 6ms/step - loss: 3.5149 - mape: 9.4789 - val_loss: 0.0893 - val_mape: 1.4957 - lr: 0.0010
Epoch 6/100
1154/1154 [==============================] - 5s 5ms/step - loss: 2.8764 - mape: 8.5761 - val_loss: 0.1138 - val_mape: 1.7339 - lr: 0.0010
Epoch 7/100
1154/1154 [==============================] - 8s 7ms/step - loss: 2.3201 - mape: 7.

In [ ]:
frame_history_df = pd.DataFrame.from_dict(history.history)
frame_history_df[frame_history_df["val_mape"] == frame_history_df["val_mape"].min()]
frame_history_df.to_csv("models_log.csv", index=False)
model = keras.models.load_model("models/model.keras")
y_pred_fcl = model.predict(X_trainV_norm, verbose=1).reshape(-1,)

3075/3075 [==============================] - 10s 3ms/step


In [ ]:
print(y_pred_fcl)

[15.635322 15.651991 15.669962 ... 15.948352 16.322342 15.514016]


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

a = 98398

#train (new)
X_trainn = X_train[:-a]
y_trainn = y_train[:-a]
#valid
X_trainV = X_train[-a:]
y_trainV = y_train[-a:]

# Train rf model
rf_regressor = RandomForestRegressor(n_estimators=400, max_depth=15)
rf_regressor.fit(X_trainn, y_trainn)

# Make predictions on holdout set
y_pred_rf = rf_regressor.predict(X_trainV)

# Stack predictions
X_train_stack = np.vstack((y_pred_rf, y_pred_fcl)).T

# Train linear regression model on stacked predictions
lr = LinearRegression()
lr.fit(X_train_stack, y_trainV)

# Make predictions on test set
y_pred_rf_test = rf_regressor.predict(X_test)
y_pred_fcl_test = fcl_model.predict(X_test)
X_test_stack = np.vstack((y_pred_rf_test, y_pred_fcl_test)).T
y_pred_final = lr.predict(X_test_stack)

print("Final predictions:", y_pred_final)

Final predictions: [15.52498669 15.39261411 15.65796755 ... 15.24069476 15.51345971
 16.42257086]


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, np.exp(y_pred_final))

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, np.exp(y_pred_final))

# Calculate Root Mean Squared Log Error (MAPE)
rmsle = np.sqrt(np.mean(y_pred_final - np.log(y_test))**2.0)

print("Mean Absolute Error: ", mae)
print("Mean Absolute Percentage Error: ", mape)
print("Root Mean Squared Log Error: ", rmsle)

Mean Absolute Error:  1356486.2019733414
Mean Absolute Percentage Error:  0.11200437826812204
Root Mean Squared Log Error:  0.050358097188733915


In [ ]:
#RF
#Base and Meta

import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Define the base model (Random Forest Regressor)
base_model = RandomForestRegressor(n_estimators=400, max_depth=15)

# Define the meta model (Fully Connected layer)
meta_model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', batch_size=256,  shuffle=True)

# Train the base model
base_model.fit(X_train, y_train)

# Get the predictions from the base model
base_pred = base_model.predict(X_train)

# Create a new dataset with the predictions from the base model as input features
X_new = np.concatenate((X_train, base_pred.reshape(-1, 1)), axis=1)

# Train the meta model
meta_model.fit(X_new, y_train)

# Make predictions on the test set using the ensemble model
y_pred = meta_model.predict(np.concatenate((X_test, base_model.predict(X_test).reshape(-1, 1)), axis=1))

# Evaluate the ensemble model
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, np.exp(y_pred))

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, np.exp(y_pred))

# Calculate Root Mean Squared Log Error (MAPE)
rmsle = np.sqrt(np.mean(y_pred - np.log(y_test))**2.0)

print("Mean Absolute Error: ", mae)
print("Mean Absolute Percentage Error: ", mape)
print("Root Mean Squared Log Error: ", rmsle)


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

a = 98398

#train (new)
X_trainn = X_train[:-a]
y_trainn = y_train[:-a]
#valid
X_trainV = X_train[-a:]
y_trainV = y_train[-a:]

# Train XGBoost model
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', n_estimators = 1000, max_depth = 10, learning_rate = 0.1, enable_categorical = True)
xgb_regressor.fit(X_trainn, y_trainn)


# Make predictions on holdout set
y_pred_xgb = xgb_regressor.predict(X_trainV )


# Stack predictions
X_train_stack = np.vstack((y_pred_xgb, y_pred_fcl)).T

# Train linear regression model on stacked predictions
lr = LinearRegression()
lr.fit(X_train_stack, y_trainV)

# Make predictions on test set
y_pred_xgb_test = xgb_regressor.predict(X_test)
y_pred_fcl_test = fcl_model.predict(X_test)
X_test_stack = np.vstack((y_pred_xgb_test, y_pred_fcl_test)).T
y_pred_final = lr.predict(X_test_stack)

print("Final predictions:", y_pred_final)

Final predictions: [15.4906256  15.40068789 15.71414104 ... 15.21405226 15.56231596
 16.39327844]


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, np.exp(y_pred_final))

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, np.exp(y_pred_final))

# Calculate Root Mean Squared Log Error (MAPE)
rmsle = np.sqrt(np.mean(y_pred_final - np.log(y_test))**2.0)

print("Mean Absolute Error: ", mae)
print("Mean Absolute Percentage Error: ", mape)
print("Root Mean Squared Log Error: ", rmsle)

Mean Absolute Error:  1250986.2303879734
Mean Absolute Percentage Error:  0.10379771787990201
Root Mean Squared Log Error:  0.051947632245624056


In [ ]:
#Base and Meta XGBoost

import numpy as np
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Define the base model (XGBoost Regressor)
base_model = XGBRegressor(objective='reg:squarederror', n_estimators = 1000, max_depth = 10, learning_rate = 0.1, enable_categorical = True)

# Define the meta model (Fully Connected layer)
meta_model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', batch_size=256, shuffle=True)

# Train the base model
base_model.fit(X_train, y_train)

# Get the predictions from the base model
base_pred = base_model.predict(X_train)

# Create a new dataset with the predictions from the base model as input features
X_new = np.concatenate((X_train, base_pred.reshape(-1, 1)), axis=1)

# Train the meta model
meta_model.fit(X_new, y_train)

# Make predictions on the test set using the ensemble model
y_pred = meta_model.predict(np.concatenate((X_test, base_model.predict(X_test).reshape(-1, 1)), axis=1))

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, np.exp(y_pred))

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, np.exp(y_pred))

# Calculate Root Mean Squared Log Error (MAPE)
rmsle = np.sqrt(np.mean(y_pred - np.log(y_test))**2.0)

print("Mean Absolute Error: ", mae)
print("Mean Absolute Percentage Error: ", mape)
print("Root Mean Squared Log Error: ", rmsle)


Mean Absolute Error:  1199830.0709619669
Mean Absolute Percentage Error:  0.10021235260663525
Root Mean Squared Log Error:  0.0725534274328916


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

# Загрузка данных и подготовка к обучению
# X - входные данные, y - целевые данные

# Параметры для полностью связанного слоя
hidden_layer_sizes = (100,)
alpha = 0.01

# Параметры для XGBoost Regressor
n_estimators = 100
learning_rate = 0.1

# Обучение полностью связанного слоя
mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, random_state=1)
mlp_regressor.fit(X_train, y_train)


X_train_old: (X_train (new), X_valid), X_test

#тест: класс алг, нн, стекинг

X_train[:-a] - train (new)
X_train[-a:] - valid


# Обучение XGBoost Regressor
xgb_regressor = XGBRegressor(objective='reg:squarederror',
                             n_estimators=1000,
                             max_depth=10,
                             learning_rate = 0.1,
                             enable_categorical=True)

xgb_regressor.fit(X_train[:-a], y_train[:-a])

y_pred_xgb = xgb_regressor.predict(X_train[-a:])

y_pred_nn = model_nn.predict(X_train[-a:])

X_train_stack = np.vstack((y_pred_xgb, y_pred_nn))

lr.fit(X_train_stack, y_valid)

y_pred_xgb_test = xgb_regressor.predict(X_test)
y_pred_nn_test = model_nn.predict(X_test)
X_test_stack = np.vstack((y_pred_xgb_test, y_pred_nn_test))
lr.predict(X_test_stack)

простой стекинг: 0.5*y_pred_xgb + 0.5*y_pred_nn

class Model(sklearn....):
  self.model = keras.model

  def predict(X):
    return self.model.predict(X).reshape(-1,)

# Создание ансамбля стекинга
stacking_regressor = StackingRegressor(estimators=[('nn', keras_model), ('xgb', xgb_regressor)], final_estimator=LinearRegression())

# Обучение ансамбля стекинга
stacking_regressor.fit(X_train, y_train)

# Прогнозирование с помощью ансамбля стекинга
y_pred = stacking_regressor.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = np.mean((np.exp(y_pred) - y_test)**2)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((y_test - np.exp(y_pred)) / y_test)) * 100

# Calculate Root Mean Squared Log Error (MAPE)
rmsle = np.sqrt(mean_squared_log_error(np.log(y_test) , np.abs(y_pred)))


print("Mean Absolute Error: ", mae)
print("Mean Absolute Percentage Error: ", mape)
print("Root Mean Squared Log Error: ", rmsle)


Mean Absolute Error:  4884017266241.521
Mean Absolute Percentage Error:  9.089926015421058
Root Mean Squared Log Error:  0.008798107361659386
